In [1]:
using ModelingToolkit, DifferentialEquations

@parameters L_PFK L_PFK2 Vf_PFK Vr_PFK Vf_PFK2 Vr_PFK2 Vf_FBP Vr_FBP K_f6p K_ATP K_f16bp K_ADP K_cit K_AMP K_f26bp K_PFK2 K2_f6p K2_ATP K2_f26bp K2_ADP K_FBP_f26bp K_FBP_f6p K_Pi

@parameters C_ATP C_ADP C_cit C_AMP C_Pi C_G6P C_GAP C_MgATP C_MgADP C_Mg C_DHAP
@parameters Vf_GPI Vr_GPI Kf_GPI Kr_GPI 
@parameters Vf_ALD Vr_ALD KAld_f16bp K_GAP K_DHAP
@parameters f26bp
@variables t f6p(t) f16bp(t)
D = Differential(t)

r_GPI = (Vf_GPI*C_G6P/Kf_GPI - Vr_GPI*f6p/Kr_GPI)/
    (1 + C_G6P/Kf_GPI + f6p/Kr_GPI)

N_PFK = 1 + L_PFK*(1+C_ATP/K_ATP)^4*(1+C_cit/K_cit)/
    ((1 + f6p/K_f6p + f16bp/K_f16bp)^4 + (1 + C_AMP/K_AMP)^4 * (1 + f26bp/K_f26bp)^4)

r_PFK = ((Vf_PFK*C_ATP*f6p/(K_f6p*K_ATP) - Vr_PFK*C_ADP*f16bp/(K_f16bp*K_ADP))/
    ((1 + f6p/K_f6p)*(1 + C_ATP/K_ATP) + (1 + f16bp/K_f16bp)*(1 + C_ADP/K_ADP) -1)) *
    (1/N_PFK)

r_ALD = Vf_ALD*f16bp/KAld_f16bp - Vr_ALD*C_GAP*C_DHAP/K_GAP*K_DHAP  / (1 + f6p/KAld_f16bp + C_GAP/K_GAP)

eqs = [D(f6p) ~ r_GPI - r_PFK,
    D(f16bp) ~ r_PFK - r_ALD]

@named sys = ODESystem(eqs, t)

sys=structural_simplify(sys)

Model sys with 2 equations
Unknowns (2):
  f6p(t)
  f16bp(t)
Parameters (27):
  Vf_GPI
  K_f16bp
  K_AMP
  K_ATP
⋮
Incidence matrix:2×4 SparseArrays.SparseMatrixCSC{Num, Int64} with 6 stored entries:
 ×  ×  ×  ⋅
 ×  ×  ⋅  ×

In [ ]:
u0 = [f6p => 10,
    f16bp => 0.1,
    f26bp => 0.1]

p1 = [L_PFK => 2*10^-3,
    L_PFK2 => 1*10^-3]

V = [Vf_GPI => 4.8*10^4,
    Vr_GPI => 40*10^4,
    Vf_PFK => 15.5*10^2,
    Vr_PFK => 6.78*10^1,
    Vf_PFK2 => 41.6, 
    Vr_PFK2 => 30, #
    Vf_FBP => 40, #
    Vr_FBP => 30, #
    Vf_ALD => 6.75*10^2,
    Vr_ALD => 2.32*10^3]

K = [Kf_GPI => 0.3,
    Kr_GPI => 0.123,
    KAld_f16bp => 5*10^-2,
    K_GAP => 0.189,
    K_f6p => 6*10^-2,
    K_ATP => 0.1, #or 6.8*10^-2
    K_f16bp => 0.65,
    K_ADP => 0.54,
    K_cit => 0.1, #
    K_AMP => 0.3,
    K_f26bp => 5.5*10^-3,
    K_PFK2 => 16,
    K2_f6p => 5*10^-2,
    K2_ATP => 0.15,
    K2_f26bp => 5.5*10^-3,
    K2_ADP => 0.23,
    K_FBP_f26bp => 0.1,
    K_FBP_f6p => 0.1]    

Cons = [C_ATP => 1.59*10^-1, #Mulq
    C_MgATP => 1.52, #Mulq
    C_ADP => 9.37*10^-2, #Mulq
    C_MgADP => 0.11, #Mulq
    C_Mg => 0.4, #Mulq
    C_cit => 0.1,
    C_AMP => 3*10^-2, #Mulq
    C_Pi => 1, #Mulq
    C_G6P => 3.9*10^-2, #Mulq
    C_GAP => 5.7*10^-3  #Mulq
    ]    

p= [p1
    V
    K
    Cons]


In [ ]:
tspan = (0.0, 100)
prob = ODEProblem(sys, u0, tspan, p, jac = true)
sol = solve(prob)
print(sol[end])
using Plots
plot(sol)